In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score, KFold


df = pd.read_csv(r'C:\Users\Lenovo\Desktop\Ibrahim\dataEngProject\milestone1\clean_df2.csv')

df['trip_distance'] = df['trip_distance'].replace(0, 0.01)

df['price_per_m'] = df['total_amount'] / df['trip_distance']

quantiles = df['price_per_m'].quantile([0.33, 0.66])

df.head()

D:\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,lpep_pickup_datetime,lpep_dropoff_datetime,pu_location,do_location,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,...,vendor_VeriFone Inc.,store_and_fwd_flag_N,store_and_fwd_flag_Y,rate_type_Standard rate,rate_type_Negotiated fare,payment_type_Credit card,payment_type_Cash,trip_type_Street-hail,trip_type_Dispatch,price_per_m
0,2018-01-01 00:18:50,2018-01-01 00:24:39,134,159,5.0,0.70,6.0,0.5,0.5,0.0,...,1,1,0,1,0,0,1,1,0,10.428571
1,2018-01-01 00:30:26,2018-01-01 00:46:42,107,109,5.0,3.50,14.5,0.5,0.5,0.0,...,1,1,0,1,0,0,1,1,0,4.514286
2,2018-01-01 00:07:25,2018-01-01 00:19:45,109,138,1.0,2.14,10.0,0.5,0.5,0.0,...,1,1,0,1,0,0,1,1,0,5.280374
3,2018-01-01 00:32:40,2018-01-01 00:33:41,100,100,1.0,0.03,4.5,-0.5,-0.5,0.0,...,1,1,0,1,0,0,0,1,0,210.000000
4,2018-01-01 00:38:35,2018-01-01 01:08:50,100,141,1.0,5.63,21.0,0.5,0.5,0.0,...,1,1,0,1,0,0,1,1,0,3.960924


In [2]:
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])
df['day_of_week'] = df['lpep_pickup_datetime'].dt.dayofweek
df['month'] = df['lpep_pickup_datetime'].dt.month

df.drop(columns=['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount'], inplace=True)

df.head()

,pu_location,do_location,passenger_count,trip_distance,hours,"vendor_Creative Mobile Technologies, LLC",vendor_VeriFone Inc.,store_and_fwd_flag_N,store_and_fwd_flag_Y,rate_type_Standard rate,rate_type_Negotiated fare,payment_type_Credit card,payment_type_Cash,trip_type_Street-hail,trip_type_Dispatch,price_per_m,day_of_week,month
0,134,159,5.0,0.70,0,0,1,1,0,1,0,0,1,1,0,10.428571,0,1
1,107,109,5.0,3.50,0,0,1,1,0,1,0,0,1,1,0,4.514286,0,1
2,109,138,1.0,2.14,0,0,1,1,0,1,0,0,1,1,0,5.280374,0,1
3,100,100,1.0,0.03,0,0,1,1,0,1,0,0,0,1,0,210.000000,0,1
4,100,141,1.0,5.63,0,0,1,1,0,1,0,0,1,1,0,3.960924,0,1


In [3]:
def categorize_price_quantiles(row):
    if row['price_per_m'] <= quantiles[0.33]:
        return 'low'
    elif row['price_per_m'] <= quantiles[0.66]:
        return 'medium'
    else:
        return 'high'

In [4]:
df['price_category'] = df.apply(categorize_price_quantiles, axis=1)

df.head()

,pu_location,do_location,passenger_count,trip_distance,hours,"vendor_Creative Mobile Technologies, LLC",vendor_VeriFone Inc.,store_and_fwd_flag_N,store_and_fwd_flag_Y,rate_type_Standard rate,rate_type_Negotiated fare,payment_type_Credit card,payment_type_Cash,trip_type_Street-hail,trip_type_Dispatch,price_per_m,day_of_week,month,price_category
0,134,159,5.0,0.70,0,0,1,1,0,1,0,0,1,1,0,10.428571,0,1,high
1,107,109,5.0,3.50,0,0,1,1,0,1,0,0,1,1,0,4.514286,0,1,low
2,109,138,1.0,2.14,0,0,1,1,0,1,0,0,1,1,0,5.280374,0,1,low
3,100,100,1.0,0.03,0,0,1,1,0,1,0,0,0,1,0,210.000000,0,1,high
4,100,141,1.0,5.63,0,0,1,1,0,1,0,0,1,1,0,3.960924,0,1,low


In [5]:
features = df[['pu_location', 'do_location', 'passenger_count', 'trip_distance', 'store_and_fwd_flag_N', 'store_and_fwd_flag_Y', 'rate_type_Standard rate', 'rate_type_Negotiated fare', 'hours', 'vendor_Creative Mobile Technologies, LLC', 'vendor_VeriFone Inc.', 'payment_type_Credit card', 'payment_type_Cash', 'trip_type_Street-hail', 'trip_type_Dispatch', 'day_of_week', 'month']]
label = df['price_category']
label = label.astype('category').cat.codes

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.3, random_state=42)
#y_pred = clf.predict(X_test)


# Decision Tree Classification

In [22]:
from sklearn.tree import DecisionTreeClassifier

In [32]:
Criterions = ['gini', 'entropy', 'log_loss']

accuracy = []
params = []
cross_val = []

for criterion in Criterions:

    parameters = (criterion)
    print("parameters: ", parameters)
    params.append(parameters)

    clf = DecisionTreeClassifier(criterion=criterion, max_depth= 10, min_samples_split= 10)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    accuracy.append(accuracy_score(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(clf, features, label, cv=kf, scoring='accuracy')
    cross_val.append(scores.mean())
    print(f"Cross-validation scores: {scores}")
    print(f"Mean accuracy: {scores.mean()}")
    print(f"Standard deviation: {scores.std()}")

    y_pred = clf.predict(X_train)
    print("Accuracy test on train:", accuracy_score(y_train, y_pred))

    print(" ")

print("highest accuracy: ", max(accuracy))

max_index = accuracy.index(max(accuracy))
best_param = params[max_index]
print("cross validation score", cross_val[max_index])
print("best parameters accuracy: ", best_param)


clf = DecisionTreeClassifier(criterion=best_param, max_depth= 10, min_samples_split= 10)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_train)
print("Accuracy:", accuracy_score(y_train, y_pred))
print("Classification Report:\n", classification_report(y_train, y_pred))

print("lowest accuracy: ", min(accuracy))

min_index = accuracy.index(min(accuracy))
worst_param = params[min_index]
print("cross validation score", cross_val[min_index])
print("worst parameters: ", worst_param)

clf = DecisionTreeClassifier(criterion=worst_param, max_depth= 10, min_samples_split= 10)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_train)
print("Accuracy:", accuracy_score(y_train, y_pred))
print("Classification Report:\n", classification_report(y_train, y_pred))

parameters:  gini
Accuracy: 0.8220129656637526
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.84      0.87     80841
           1       0.88      0.84      0.86     78164
           2       0.71      0.78      0.75     78237

    accuracy                           0.82    237242
   macro avg       0.83      0.82      0.82    237242
weighted avg       0.83      0.82      0.82    237242

Cross-validation scores: [0.82333824 0.82194093 0.821903   0.8228514  0.82178287]
Mean accuracy: 0.822363288041932
Standard deviation: 0.0006190201868315179
Accuracy test on train: 0.8257361854025649
 
parameters:  entropy
Accuracy: 0.8205629694573473
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.84      0.87     80841
           1       0.88      0.84      0.86     78164
           2       0.71      0.79      0.74     78237

    accuracy                           0.82    237242


### Scaling

In [28]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

features = df[['pu_location', 'do_location', 'passenger_count', 'trip_distance', 'store_and_fwd_flag_N', 'store_and_fwd_flag_Y', 'rate_type_Standard rate', 'rate_type_Negotiated fare', 'hours', 'vendor_Creative Mobile Technologies, LLC', 'vendor_VeriFone Inc.', 'payment_type_Credit card', 'payment_type_Cash', 'trip_type_Street-hail', 'trip_type_Dispatch', 'day_of_week', 'month']]

label = df['price_category']
label = label.astype('category').cat.codes

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(features_scaled, label, test_size=0.3, random_state=42)

In [29]:
Criterions = ['gini', 'entropy', 'log_loss']

accuracy = []
params = []
cross_val = []

for criterion in Criterions:

    parameters = (criterion)
    print("parameters: ", parameters)
    params.append(parameters)

    clf = DecisionTreeClassifier(criterion=criterion, max_depth= 10, min_samples_split= 10)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    accuracy.append(accuracy_score(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(clf, features, label, cv=kf, scoring='accuracy')
    cross_val.append(scores.mean())
    print(f"Cross-validation scores: {scores}")
    print(f"Mean accuracy: {scores.mean()}")
    print(f"Standard deviation: {scores.std()}")

    y_pred = clf.predict(X_train)
    print("Accuracy test on train:", accuracy_score(y_train, y_pred))

    print(" ")

print("highest accuracy: ", max(accuracy))

max_index = accuracy.index(max(accuracy))
best_param = params[max_index]
print("cross validation score", cross_val[max_index])
print("best parameters accuracy: ", best_param)


clf = DecisionTreeClassifier(criterion=best_param, max_depth= 10, min_samples_split= 10)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_train)
print("Accuracy:", accuracy_score(y_train, y_pred))
print("Classification Report:\n", classification_report(y_train, y_pred))

print("lowest accuracy: ", min(accuracy))

min_index = accuracy.index(min(accuracy))
worst_param = params[min_index]
print("cross validation score", cross_val[min_index])
print("worst parameters: ", worst_param)

clf = DecisionTreeClassifier(criterion=worst_param, max_depth= 10, min_samples_split= 10)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_train)
print("Accuracy:", accuracy_score(y_train, y_pred))
print("Classification Report:\n", classification_report(y_train, y_pred))

parameters:  gini
Accuracy: 0.8220003203479991
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.84      0.87     80841
           1       0.88      0.84      0.86     78164
           2       0.71      0.78      0.75     78237

    accuracy                           0.82    237242
   macro avg       0.83      0.82      0.82    237242
weighted avg       0.83      0.82      0.82    237242

Cross-validation scores: [0.82333824 0.82194093 0.82191564 0.8228514  0.82178287]
Mean accuracy: 0.8223658171104127
Standard deviation: 0.0006171574873950873
Accuracy test on train: 0.8257361854025649
 
parameters:  entropy
Accuracy: 0.8205587543520961
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.84      0.87     80841
           1       0.88      0.84      0.86     78164
           2       0.71      0.79      0.74     78237

    accuracy                           0.82    237242

# Random Forest Classifiaction

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [37]:
estimators = np.arange(20, 200, 20)
Criterions = ['gini', 'entropy', 'log_loss']

accuracy = []
params = []
cross_val = []

for n_est in estimators:
    for criterion in Criterions:

        parameters = (n_est, criterion)
        print("parameters: ", parameters)
        params.append(parameters)

        rf = RandomForestClassifier(n_estimators=n_est, criterion=criterion,max_depth= 10, n_jobs=-1)
        rf.fit(X_train, y_train)

        y_pred = rf.predict(X_test)
        accuracy.append(accuracy_score(y_test, y_pred))
        print("Accuracy:", accuracy_score(y_test, y_pred))
        print("Classification Report:\n", classification_report(y_test, y_pred))

        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        scores = cross_val_score(rf, features, label, cv=kf, scoring='accuracy')
        cross_val.append(scores.mean())
        print(f"Cross-validation scores: {scores}")
        print(f"Mean accuracy: {scores.mean()}")
        print(f"Standard deviation: {scores.std()}")

        y_pred = rf.predict(X_train)
        print("Accuracy test on train:", accuracy_score(y_train, y_pred))

        print(" ")
            
print("highest accuracy: ", max(accuracy))

max_index = accuracy.index(max(accuracy))
best_param = params[max_index]
print("cross validation score", cross_val[max_index])
print("best parameters accuracy: ", best_param)

parameters:  (20, 'gini')
Accuracy: 0.8205798298783521
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.83      0.86     80841
           1       0.85      0.87      0.86     78164
           2       0.71      0.77      0.74     78237

    accuracy                           0.82    237242
   macro avg       0.82      0.82      0.82    237242
weighted avg       0.83      0.82      0.82    237242

Cross-validation scores: [0.82058156 0.81934864 0.81901354 0.82107473 0.81815365]
Mean accuracy: 0.8196344231510929
Standard deviation: 0.001061262950989165
Accuracy test on train: 0.8220401291271273
 
parameters:  (20, 'entropy')
Accuracy: 0.8186155908313031
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.84      0.86     80841
           1       0.86      0.86      0.86     78164
           2       0.72      0.75      0.73     78237

    accuracy                          

In [38]:
print("lowest accuracy: ", min(accuracy))

min_index = accuracy.index(min(accuracy))
worst_param = params[min_index]
print("cross validation score", cross_val[min_index])
print("worst parameters: ", worst_param)

lowest accuracy:  0.8115215686935703
cross validation score 0.8161025790175834
worst parameters:  (20, 'log_loss')


### Scaled

In [26]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

features = df[['pu_location', 'do_location', 'passenger_count', 'trip_distance', 'store_and_fwd_flag_N', 'store_and_fwd_flag_Y', 'rate_type_Standard rate', 'rate_type_Negotiated fare', 'hours', 'vendor_Creative Mobile Technologies, LLC', 'vendor_VeriFone Inc.', 'payment_type_Credit card', 'payment_type_Cash', 'trip_type_Street-hail', 'trip_type_Dispatch', 'day_of_week', 'month']]

label = df['price_category']
label = label.astype('category').cat.codes

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(features_scaled, label, test_size=0.3, random_state=42)

In [27]:
estimators = np.arange(20, 200, 20)
Criterions = ['gini', 'entropy', 'log_loss']

accuracy = []
params = []
cross_val = []

for n_est in estimators:
    for criterion in Criterions:

        parameters = (n_est, criterion)
        print("parameters: ", parameters)
        params.append(parameters)

        rf = RandomForestClassifier(n_estimators=n_est, criterion=criterion,max_depth= 10, n_jobs=-1)
        rf.fit(X_train, y_train)

        y_pred = rf.predict(X_test)
        accuracy.append(accuracy_score(y_test, y_pred))
        print("Accuracy:", accuracy_score(y_test, y_pred))
        print("Classification Report:\n", classification_report(y_test, y_pred))

        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        scores = cross_val_score(rf, features, label, cv=kf, scoring='accuracy')
        cross_val.append(scores.mean())
        print(f"Cross-validation scores: {scores}")
        print(f"Mean accuracy: {scores.mean()}")
        print(f"Standard deviation: {scores.std()}")

        y_pred = rf.predict(X_train)
        print("Accuracy test on train:", accuracy_score(y_train, y_pred))

        print(" ")
            
print("highest accuracy: ", max(accuracy))

max_index = accuracy.index(max(accuracy))
best_param = params[max_index]
print("cross validation score", cross_val[max_index])
print("best parameters accuracy: ", best_param)

parameters:  (20, 'gini')
Accuracy: 0.8196820124598512
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.82      0.86     80841
           1       0.86      0.86      0.86     78164
           2       0.71      0.78      0.74     78237

    accuracy                           0.82    237242
   macro avg       0.82      0.82      0.82    237242
weighted avg       0.83      0.82      0.82    237242

Cross-validation scores: [0.81920954 0.81881753 0.82012633 0.8202591  0.81778062]
Mean accuracy: 0.8192386239338395
Standard deviation: 0.0009092133093224404
Accuracy test on train: 0.8213988290402358
 
parameters:  (20, 'entropy')
Accuracy: 0.8142698173173384
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.85      0.86     80841
           1       0.85      0.87      0.86     78164
           2       0.72      0.72      0.72     78237

    accuracy                         

# KNN Classification

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [9]:
features = df[['pu_location', 'do_location', 'passenger_count', 'trip_distance', 'store_and_fwd_flag_N', 'store_and_fwd_flag_Y', 'rate_type_Standard rate', 'rate_type_Negotiated fare', 'hours', 'vendor_Creative Mobile Technologies, LLC', 'vendor_VeriFone Inc.', 'payment_type_Credit card', 'payment_type_Cash', 'trip_type_Street-hail', 'trip_type_Dispatch', 'day_of_week', 'month']]

label = df['price_category']
label = label.astype('category').cat.codes

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(features_scaled, label, test_size=0.3, random_state=42)

In [32]:
nNeighbours = [5, 10, 20, 30, 40, 50, 100, 150, 200, 250, 300, 350, 400, 450,500, 550, 600, 650, 700, 750, 800, 850, 900, 1000, 1500]

params = []
accuracy = []
for n in nNeighbours:
    parameters = (n)
    params.append(parameters)
    print(f"parameters: {parameters}")

    knn = KNeighborsClassifier(n_neighbors=n, n_jobs=-1)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    
    accuracy.append(accuracy_score(y_test, y_pred))

    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    
    

print("highest accuracy: ", max(accuracy))

max_index = accuracy.index(max(accuracy))
best_param = params[max_index]
print("best parameters accuracy: ", best_param)

print("lowest accuracy: ", min(accuracy))

min_index = accuracy.index(min(accuracy))
worst_param = params[min_index]
print("worst parameters: ", worst_param)

parameters: 5
Accuracy: 0.7839716407718701
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.84      0.83     80841
           1       0.85      0.83      0.84     78164
           2       0.68      0.68      0.68     78237

    accuracy                           0.78    237242
   macro avg       0.78      0.78      0.78    237242
weighted avg       0.78      0.78      0.78    237242

parameters: 10
Accuracy: 0.7934429822712673
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.87      0.84     80841
           1       0.86      0.85      0.85     78164
           2       0.70      0.66      0.68     78237

    accuracy                           0.79    237242
   macro avg       0.79      0.79      0.79    237242
weighted avg       0.79      0.79      0.79    237242

parameters: 20
Accuracy: 0.7961237892110166
Classification Report:
               precision    recall  

In [34]:
knn = KNeighborsClassifier(n_neighbors=best_param, n_jobs=-1)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_train)
print("Accuracy test on train for best parameters:", accuracy_score(y_train, y_pred))

knn = KNeighborsClassifier(n_neighbors=worst_param, n_jobs=-1)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_train)
print("Accuracy test on train for worst parameters:", accuracy_score(y_train, y_pred))

Accuracy test on train for best parameters: 0.8178238068657046


MemoryError: Unable to allocate 4.22 MiB for an array with shape (553563,) and data type float64

In [8]:
nNeighbours = [5, 10, 20, 30, 40, 50, 100, 150, 200, 250, 300, 350, 400, 450,500, 550, 600, 650, 700, 750, 800, 850, 900, 1000, 1500]

params = []
accuracy = []
for n in nNeighbours:
    parameters = (n)
    params.append(parameters)
    print(f"parameters: {parameters}")

    knn = KNeighborsClassifier(n_neighbors=n, weights = 'distance', n_jobs=-1)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    
    accuracy.append(accuracy_score(y_test, y_pred))

    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))

#     kf = KFold(n_splits=5, shuffle=True, random_state=42)
#     scores = cross_val_score(knn, features_scaled, label, cv=kf, scoring='accuracy')
#     accuracy.append(scores.mean())
#     print(f"Cross-validation scores: {scores}")
#     print(f"Mean accuracy: {scores.mean()}")
#     print(f"Standard deviation: {scores.std()}")

print("highest accuracy: ", max(accuracy))

max_index = accuracy.index(max(accuracy))
best_param = params[max_index]
print("best parameters accuracy: ", best_param)

print("lowest accuracy: ", min(accuracy))

min_index = accuracy.index(min(accuracy))
worst_param = params[min_index]
print("worst parameters: ", worst_param)

parameters: 5
Accuracy: 0.7845659706122862
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.83      0.83     80841
           1       0.85      0.83      0.84     78164
           2       0.68      0.69      0.68     78237

    accuracy                           0.78    237242
   macro avg       0.79      0.78      0.78    237242
weighted avg       0.79      0.78      0.79    237242

parameters: 10
Accuracy: 0.7946948685308672
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.84      0.84     80841
           1       0.87      0.83      0.85     78164
           2       0.69      0.71      0.70     78237

    accuracy                           0.79    237242
   macro avg       0.80      0.79      0.80    237242
weighted avg       0.80      0.79      0.80    237242

parameters: 20
Accuracy: 0.7998541573583092
Classification Report:
               precision    recall  

In [10]:
# nNeighbours = [5, 10, 20, 30, 40, 50, 100, 150, 200, 250, 300, 350, 400, 450,500, 550, 600, 650, 700, 750, 800, 850, 900, 1000, 1500]
metric = ['euclidean', 'manhattan', 'chebyshev', 'cosine', 'hamming']
params = []
accuracy = []
for m in metric:
    parameters = (m)
    params.append(parameters)
    print(f"parameters: {parameters}")

    knn = KNeighborsClassifier(n_neighbors=30, weights = 'distance', metric = m, n_jobs=-1)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    
    accuracy.append(accuracy_score(y_test, y_pred))

    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))

#     kf = KFold(n_splits=5, shuffle=True, random_state=42)
#     scores = cross_val_score(knn, features_scaled, label, cv=kf, scoring='accuracy')
#     accuracy.append(scores.mean())
#     print(f"Cross-validation scores: {scores}")
#     print(f"Mean accuracy: {scores.mean()}")
#     print(f"Standard deviation: {scores.std()}")

print("highest accuracy: ", max(accuracy))

max_index = accuracy.index(max(accuracy))
best_param = params[max_index]
print("best parameters accuracy: ", best_param)

print("lowest accuracy: ", min(accuracy))

min_index = accuracy.index(min(accuracy))
worst_param = params[min_index]
print("worst parameters: ", worst_param)

parameters: euclidean
Accuracy: 0.8007688351978149
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.85      0.84     80841
           1       0.88      0.82      0.85     78164
           2       0.69      0.73      0.71     78237

    accuracy                           0.80    237242
   macro avg       0.80      0.80      0.80    237242
weighted avg       0.80      0.80      0.80    237242

parameters: manhattan
Accuracy: 0.806025071446034
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.86      0.85     80841
           1       0.88      0.83      0.85     78164
           2       0.70      0.73      0.72     78237

    accuracy                           0.81    237242
   macro avg       0.81      0.81      0.81    237242
weighted avg       0.81      0.81      0.81    237242

parameters: chebyshev
Accuracy: 0.7880265720235035
Classification Report:
               

### unscaled

In [11]:
features = df[['pu_location', 'do_location', 'passenger_count', 'trip_distance', 'store_and_fwd_flag_N', 'store_and_fwd_flag_Y', 'rate_type_Standard rate', 'rate_type_Negotiated fare', 'hours', 'vendor_Creative Mobile Technologies, LLC', 'vendor_VeriFone Inc.', 'payment_type_Credit card', 'payment_type_Cash', 'trip_type_Street-hail', 'trip_type_Dispatch', 'day_of_week', 'month']]

label = df['price_category']
label = label.astype('category').cat.codes

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.3, random_state=42)

In [19]:
knn = KNeighborsClassifier(n_neighbors=30, weights = 'distance', leaf_size= 20, metric = 'manhattan', n_jobs=-1)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(knn, features, label, cv=kf, scoring='accuracy')
accuracy.append(scores.mean())
print(f"Cross-validation scores: {scores}")
print(f"Mean accuracy: {scores.mean()}")
print(f"Standard deviation: {scores.std()}")

y_pred = knn.predict(X_train)

print("Accuracy test on train:", accuracy_score(y_train, y_pred))

Accuracy: 0.7714527781758711
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.83      0.81     80841
           1       0.87      0.80      0.83     78164
           2       0.66      0.68      0.67     78237

    accuracy                           0.77    237242
   macro avg       0.77      0.77      0.77    237242
weighted avg       0.77      0.77      0.77    237242

Cross-validation scores: [0.7751342  0.77353456 0.77358514 0.7729023  0.77377482]
Mean accuracy: 0.7737862051959712
Standard deviation: 0.0007351545748557183
Accuracy test on train: 0.9968422745017279


# Naive Bayes Calssification

In [8]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

features = df[['pu_location', 'do_location', 'passenger_count', 'trip_distance', 'store_and_fwd_flag_N', 'store_and_fwd_flag_Y', 'rate_type_Standard rate', 'rate_type_Negotiated fare', 'hours', 'vendor_Creative Mobile Technologies, LLC', 'vendor_VeriFone Inc.', 'payment_type_Credit card', 'payment_type_Cash', 'trip_type_Street-hail', 'trip_type_Dispatch', 'day_of_week', 'month']]
label = df['price_category']
label = label.astype('category').cat.codes

# scaler = StandardScaler()
# features_scaled = scaler.fit_transform(features)

# # Apply PCA
# pca = PCA()
# principal_components = pca.fit_transform(features)
# features_pca = pd.DataFrame(data=principal_components)

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.3)#, random_state=42)

## Gaussian Naive Bayes

In [18]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.49390495780679644
Classification Report:
               precision    recall  f1-score   support

           0       0.37      0.00      0.00     80664
           1       0.91      0.52      0.66     78298
           2       0.40      0.98      0.56     78280

    accuracy                           0.49    237242
   macro avg       0.56      0.50      0.41    237242
weighted avg       0.56      0.49      0.41    237242



In [19]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(gnb, features, label, cv=kf, scoring='accuracy')
print(f"Cross-validation scores: {scores}")
print(f"Mean accuracy: {scores.mean()}")
print(f"Standard deviation: {scores.std()}")

Cross-validation scores: [0.49592504 0.49232744 0.49569742 0.49503354 0.49405353]
Mean accuracy: 0.49460739373170376
Standard deviation: 0.001312319719645973


## Multinomial Naive Bayes

In [20]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred = mnb.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6754579711855405
Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.78      0.75     80664
           1       0.81      0.68      0.74     78298
           2       0.53      0.56      0.54     78280

    accuracy                           0.68    237242
   macro avg       0.68      0.67      0.68    237242
weighted avg       0.68      0.68      0.68    237242



In [21]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(mnb, features, label, cv=kf, scoring='accuracy')
print(f"Cross-validation scores: {scores}")
print(f"Mean accuracy: {scores.mean()}")
print(f"Standard deviation: {scores.std()}")

Cross-validation scores: [0.6761591  0.67512851 0.67600736 0.67347197 0.67252989]
Mean accuracy: 0.6746593660889852
Standard deviation: 0.0014304526244244514


## Bernoulli Naive Bayes

In [22]:
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
bnb.fit(X_train, y_train)
y_pred = bnb.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.3587644683487747
Classification Report:
               precision    recall  f1-score   support

           0       0.36      0.56      0.44     80664
           1       0.40      0.12      0.19     78298
           2       0.35      0.39      0.37     78280

    accuracy                           0.36    237242
   macro avg       0.37      0.36      0.33    237242
weighted avg       0.37      0.36      0.33    237242



In [23]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(bnb, features, label, cv=kf, scoring='accuracy')
print(f"Cross-validation scores: {scores}")
print(f"Mean accuracy: {scores.mean()}")
print(f"Standard deviation: {scores.std()}")

Cross-validation scores: [0.35666188 0.35869778 0.35669982 0.35941857 0.35876733]
Mean accuracy: 0.3580490765738709
Standard deviation: 0.001145165447656292


## Complement Naive Bayes

In [24]:
from sklearn.naive_bayes import ComplementNB

cpnb = ComplementNB()
cpnb.fit(X_train, y_train)
y_pred = cpnb.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6190387873985214
Classification Report:
               precision    recall  f1-score   support

           0       0.56      0.94      0.70     80664
           1       0.71      0.84      0.77     78298
           2       0.57      0.07      0.12     78280

    accuracy                           0.62    237242
   macro avg       0.61      0.62      0.53    237242
weighted avg       0.61      0.62      0.53    237242



In [25]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(cpnb, features, label, cv=kf, scoring='accuracy')
print(f"Cross-validation scores: {scores}")
print(f"Mean accuracy: {scores.mean()}")
print(f"Standard deviation: {scores.std()}")

Cross-validation scores: [0.62028566 0.61951429 0.61796524 0.62087999 0.61785143]
Mean accuracy: 0.6192993215773801
Standard deviation: 0.0012160388772750343


## Categorical Naive Bayes

In [26]:
from sklearn.naive_bayes import CategoricalNB

catnb = CategoricalNB()
catnb.fit(X_train, y_train)
y_pred = catnb.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7608054223113951
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.75      0.81     80664
           1       0.82      0.78      0.80     78298
           2       0.62      0.75      0.68     78280

    accuracy                           0.76    237242
   macro avg       0.78      0.76      0.76    237242
weighted avg       0.78      0.76      0.77    237242



In [27]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(catnb, features, label, cv=kf, scoring='accuracy')
print(f"Cross-validation scores: {scores}")
print(f"Mean accuracy: {scores.mean()}")
print(f"Standard deviation: {scores.std()}")

Cross-validation scores: [0.75977643 0.75935281 0.76168588 0.76007992 0.75963101]
Mean accuracy: 0.7601052092488034
Standard deviation: 0.0008244045612557889


# Logistic Regression

In [19]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet', 'None'],
    'solver': ['lbfgs', 'saga', 'sag', 'newton-cg']
}

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

log_reg = LogisticRegression(max_iter=100)

grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, cv=5, verbose=3, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best parameters found:", grid_search.best_params_)

y_pred = grid_search.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Fitting 5 folds for each of 64 candidates, totalling 320 fits


D:\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
220 fits failed out of a total of 320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "D:\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1168, in fit
    solver = _check_solver(self.solver, self.penalty, self.d

Best parameters found: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
Accuracy: 0.7942059163217305
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.87      0.85     80841
           1       0.87      0.80      0.84     78164
           2       0.69      0.70      0.69     78237

    accuracy                           0.79    237242
   macro avg       0.80      0.79      0.79    237242
weighted avg       0.80      0.79      0.79    237242



In [10]:
log_reg = LogisticRegression(C = 10, penalty= 'l2', solver= 'newton-cg', max_iter=100)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_train)
print("Accuracy:", accuracy_score(y_train, y_pred))

Accuracy: 0.7948237147352695


In [9]:
y_pred = grid_search.predict(X_train)

print("Accuracy:", accuracy_score(y_train, y_pred))
print("Classification Report:\n", classification_report(y_train, y_pred))

Accuracy: 0.7948237147352695
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.88      0.85    188017
           1       0.87      0.80      0.84    182815
           2       0.69      0.70      0.70    182731

    accuracy                           0.79    553563
   macro avg       0.80      0.79      0.79    553563
weighted avg       0.80      0.79      0.80    553563



In [20]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

features = df[['pu_location', 'do_location', 'passenger_count', 'trip_distance', 'store_and_fwd_flag_N', 'store_and_fwd_flag_Y', 'rate_type_Standard rate', 'rate_type_Negotiated fare', 'hours', 'vendor_Creative Mobile Technologies, LLC', 'vendor_VeriFone Inc.', 'payment_type_Credit card', 'payment_type_Cash', 'trip_type_Street-hail', 'trip_type_Dispatch', 'day_of_week', 'month']]

label = df['price_category']
label = label.astype('category').cat.codes

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(features_scaled, label, test_size=0.3, random_state=42)

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

log_reg = LogisticRegression(max_iter=100)

grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, cv=5, verbose=3, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best parameters found:", grid_search.best_params_)

y_pred = grid_search.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Fitting 5 folds for each of 64 candidates, totalling 320 fits


D:\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
220 fits failed out of a total of 320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "D:\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1168, in fit
    solver = _check_solver(self.solver, self.penalty, self.d

Best parameters found: {'C': 10, 'penalty': 'l2', 'solver': 'sag'}
Accuracy: 0.7942143465322329
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.88      0.85     80841
           1       0.87      0.80      0.84     78164
           2       0.69      0.70      0.69     78237

    accuracy                           0.79    237242
   macro avg       0.80      0.79      0.79    237242
weighted avg       0.80      0.79      0.79    237242



In [22]:
y_pred = grid_search.predict(X_train)

print("Accuracy:", accuracy_score(y_train, y_pred))
print("Classification Report:\n", classification_report(y_train, y_pred))

Accuracy: 0.7948201017770335
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.88      0.85    188017
           1       0.87      0.80      0.84    182815
           2       0.69      0.70      0.70    182731

    accuracy                           0.79    553563
   macro avg       0.80      0.79      0.79    553563
weighted avg       0.80      0.79      0.80    553563



# Conclusion

## Decision Tree Classifier

I tested this model using (gini, entropy, log_loss) criterions. After testing the model on the test dataset and applying cross validation for each criterion, I tried to predict the output using the train dataset and to compare the accuracy with that when predicting using the testing data to check if the model was over fitting or not. the accuracy of the prediction using the train set was much higher than that of the prediction using test set, which was indicating that there was overfitting. So I used regularization in the parameter set and forced the values of the max_depth and the min_samples_split so as to prevent overfitting while getting the best accuracy possible. Using Cross Validation, it was found that the best parameter was criterion = 'gini'. the train set accuracy with this parameter was 82.20%, the Cross Validation accuracy was 82.23%, while the train set accuracy was 82.57%. I tried afterthat to scale the data and fit the model again, but it seems like it has no effect on the accuracy of the model.

## Random Forest Classifer

The Random Forest Classifier provided similar results to those of the Decision Tree. 
the highest accuracy was 82.11% using the test set, 82.05% using Cross Validation, and 82.26% using the train set. This accuracy was achived using the parameters (n_estimators= 60, criterion= 'gini'). max_depth was also made 10 to prevent over fitting of the model. Scaling here once more did not have any recognized effect on the accuracy of the model, as with the decision tree.

## KNN Classifier

The highest accuracy that was achived by the parameters (n_neighhbours = 30, weights = 'distance', metric = 'manhattan'), and this recorded an accuracy of 77.14% using the test set, and 77.37% using CV. However, the accuracy on the train set was 99.68%, which means that the KNN model is significantly overfitting.
When applying scaling on the data using the same parameters, the accuracy increased to 79.61% on the test set, and 81.78% on the train set. which means also that the overfitting decreased after scaling the data

## Naive Bayes Classifier

For Naive Bayes, we tested two models. Namely, Gaussian Naive Bayes and Categorical Naive Bayes.
For Gaussian Naive Bayes, we observed a train set accuracy of 49.68% and a test set accuracy of 49.58%. This corresponds to a significantly low bias, where the model is severely underfit, barely performing better than a completely random guessing model (Which should achieve accuracy ~33%). We think that the reason behind the failure of this model is due to the fact that most features in our dataset are not Gaussian distributed, and that some features may be correlated / dependant, which goes against the assumption by the Naive Bayes' model.
For Categorical Naive Bayes, we were able to achieve significantly better (albeit still relatively bad!) results, with a train set accuracy of 76.39% and a test set accuracy of 76.43%.
Overall, we do NOT recommend Naive Bayes at all as a model for this set.

## Logistic Regression Classifer

Using the Logistic Regression model, the best parameters found to provide the highest accuracy was (C = 10, penalty = 'l2', solver = 'newton-cg'). These parameters provided a CV accuracy of 79.42%, the train set accuracy was 79.48%, meaning that there is no overfitting as the model uses l2 regularization scheme. I then tried to apply scaling on the data using the same parameters, but no change in the accuracy was recorded. 